In [24]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker


import pynwb
import nspike_helpers as ns 
import query_helpers as qu
import fl_apparatus as ap

mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# debug settings
limit_num_of_tets = None # To speed up testing. Set to None to load all tets

# Session-specific params
# data_dir = '/opt/data46/FrankData/kkay/Bon/'

data_dir, data_src = os.path.expanduser('~/Data/FrankData/kkay/Bon'), 'local'
# data_dir, data_src = os.path.expanduser('~/Data/FrankData/CRCNS/Bon'), 'CRCNS'

anim = 'Bon' 
day = 5 # below we'll code date as 2006-Jan-'Day'

# 'Wall clock' (i.e. actual) date and time of the Nspike time = 0 for this experiment.
# NOTE: this is not the true zero_time, as we don't have easy access to that without digging in lab notebooks.
# TODO: dig in lab notebooks, at least to get the date of recording.
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))

# General params/presets
file_create_date = datetime.now(tz.tzlocal())

eeg_samprate = 1500.0 # Hz

eeg_subdir = "EEG"
epochs_file = "times.mat"
tetinfo_file = "tetinfo.mat"
NSpike_timestamps_per_sec = 10000

In [26]:
print(file_create_date)
print(dataset_zero_time)

2019-04-11 10:30:09.527744-07:00
2006-01-05 12:00:00-08:00


### Parse inputs and create NWBfile

In [27]:
day_str = '%02d' % day

nwb_filename = anim + day_str + '_test.nwb'

# check the input arguments
if not os.path.exists(data_dir):
        print('Error: data_dir %s does not exist' % data_dir)
        exit(-1)

# get filename prefix and file locations
prefix = anim.lower()
eeg_path = os.path.join(data_dir, eeg_subdir)

# Calculate the POSIX timestamp when Nspike clock = 0 (seconds)
NSpike_posixtime_offset = dataset_zero_time.timestamp()

# We'll still store NSpike/Trodes zero time in nwbfile.session_start_time, 
# so that we can recreate the experimental timestamps, even though that
# violates the definition of session_start_time as zero time. We need some
# other way to indicate that the timestamps are POSIX (easy: if they are over
# a trillion, then they're POSIX!)

In [28]:
nwbf = pynwb.NWBFile(
           session_description='Converted NSpike data from %s' % data_dir,
           identifier=anim+day_str,
           session_start_time=dataset_zero_time,
           file_create_date=file_create_date,
           lab='Frank Laboratory',
           experimenter='Mattias Karlsson',
           institution='UCSF',
           experiment_description='Recordings from awake behaving rat',
           session_id=data_dir)

### Animal Behavior

In [29]:
# create position, direction and speed
position_list = []
pos = pynwb.behavior.Position(spatial_series=position_list, 
                              name='Position')

direction_list = []
dir = pynwb.behavior.CompassDirection(spatial_series=direction_list, 
                                      name='Head Direction')

speed_list = []
speed = pynwb.behavior.BehavioralTimeSeries(time_series=speed_list, 
                                            name='Speed')

# NOTE that day_inds is 0 based
# time_list = {}
# nwb_epoch = {}
pos_files = ns.get_files_by_day(data_dir, prefix, 'pos')
task_files = ns.get_files_by_day(data_dir, prefix, 'task')

if data_src == 'local':
    linpos_files = ns.get_files_by_day(data_dir, prefix, 'linpos')

In [30]:
mat = ns.loadmat_ff(task_files[day], 'task')
task_struct = mat[day]
# find the pos file for this day and load it

mat = ns.loadmat_ff(pos_files[day], 'pos')
pos_struct = mat[day]

# CRCNS data does not presently have linpos files
if data_src == 'local':
    mat = ns.loadmat_ff(linpos_files[day], 'linpos')
    linpos_struct = mat[day]
    # Fill in None for epochs that have no linpos data
    for k in task_struct.keys():
        if k not in linpos_struct:
            linpos_struct[k] = None

# compile a list of time intervals in the pos struct (i.e. start/stop times of position data)
# currently this is how epochs are defined in Matlab. The task struct, which contains metadata about
# epochs, does not itself have start/stop times of the epoch.
pos_time_ivls = []

# Assume field order: (time,x,y,dir,vel)
(time_idx, x_idx, y_idx, dir_idx, vel_idx) = range(5)

In [31]:
for epoch_num, pos_epoch in pos_struct.items():

    # convert times to POSIX time
    timestamps = pos_epoch['data'][:,time_idx] + NSpike_posixtime_offset

    # TODO: create a shared TimeSeries for timestamps, across all behavioral timeseries
    # ?? timestamps_obj = pynwb.TimeSeries(timestamps=timestamps...)

    # collect times of epoch start and end
    pos_time_ivls.append([timestamps[0], timestamps[-1]])

    m_per_pixel = pos_epoch['cmperpixel'][0,0]/100 # NWB wants meters per pixel

    # we can also create new SpatialSeries for the position, direction and velocity information
    #NOTE: Each new spatial series has to have a unique name.
    pos.create_spatial_series(name='Position d%d e%d' % (day, epoch_num), 
                              timestamps = timestamps,
                              data=pos_epoch['data'][:, (x_idx, y_idx)] * m_per_pixel,
                              reference_frame='corner of video frame',
                              #conversion=m_per_pixel,
                              #unit='m'
                              ) # *after* conversion

    dir.create_spatial_series(name='Head Direction d%d e%d'% (day, epoch_num), 
                              timestamps=timestamps,
                              data=pos_epoch['data'][:, dir_idx],
                              reference_frame='0=facing top of video frame (?), positive clockwise (?)',
                              #unit='radians'
                              )

    speed.create_timeseries(name='Speed d%d e%d' % (day, epoch_num),
                             timestamps=timestamps,
                             data=pos_epoch['data'][:, vel_idx] * m_per_pixel,
                             unit='m/s',
                             #conversion=m_per_pixel,
                             description='smoothed movement speed estimate')    

In [32]:
# create a Processing module for behavior
behav_mod = nwbf.create_processing_module(name='Behavior', 
                                          description='Behavioral variables')
# add the position, direction and speed data
behav_mod.add_data_interface(pos)
behav_mod.add_data_interface(dir)
behav_mod.add_data_interface(speed)


Speed <class 'pynwb.behavior.BehavioralTimeSeries'>
Fields:
  time_series: { Speed d5 e1 <class 'pynwb.base.TimeSeries'>,  Speed d5 e2 <class 'pynwb.base.TimeSeries'>,  Speed d5 e3 <class 'pynwb.base.TimeSeries'>,  Speed d5 e4 <class 'pynwb.base.TimeSeries'>,  Speed d5 e5 <class 'pynwb.base.TimeSeries'>,  Speed d5 e6 <class 'pynwb.base.TimeSeries'>,  Speed d5 e7 <class 'pynwb.base.TimeSeries'>,  Speed d5 e8 <class 'pynwb.base.TimeSeries'>,  Speed d5 e9 <class 'pynwb.base.TimeSeries'> }

In [33]:
ns_path = "franklab_apparatus.namespace.yaml"
pynwb.load_namespaces(ns_path)

('franklab_apparatus',)

### Sleep box
Define the sleep box geometry as the minimum rectangle enclosing all of the animal positions across all sleep epochs in this day.
Note that this is likely not the actual bounds of the sleep box.

In [34]:
all_sleep_pos = np.empty(shape=(0, 2))
for epoch_num, task_epoch in task_struct.items():
    if ('type' in task_epoch.keys()) and (task_epoch['type'][0] == 'sleep'):
        sleep_pos = behav_mod.data_interfaces['Position']['Position d%d e%d' % (day, epoch_num)].data
        all_sleep_pos = np.concatenate((all_sleep_pos, sleep_pos), axis=0)
min_x, min_y = np.min(all_sleep_pos, axis=0)
max_x, max_y = np.max(all_sleep_pos, axis=0)

# Polygon vertices defined in clockwise order
sleep_box_polygon = ap.PolygonNode(name='Sleep box polygon', 
                                   coords=[[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]],
                                   interior_coords=None)

# The Sleep Box apparatus is just the sleep box polygon with no edges, since it is not connected to anything else
sleep_box_appar = ap.Apparatus(name='Sleep Box', nodes=[sleep_box_polygon], edges=[])
behav_mod.add_data_interface(sleep_box_appar)

# Sleep task
description = 'The animal sleeps or wanders freely around a small, empty box.'
sleep_box_task = ap.Task(name='Sleep', description=description)
behav_mod.add_data_interface(sleep_box_task)


Sleep <class 'fl_apparatus.Task'>
Fields:
  description: The animal sleeps or wanders freely around a small, empty box.

### Epochs

In [35]:
# # ------------------------------------------------------------------
# # The start/stop time of epochs in NWB will be drawn from the start/stop times of data in pos_struct.
# # (Start/stop times of epochs are not explicitly defined in the Matlab task_struct.)
# # ------------------------------------------------------------------
epoch_tags = ''  # tags is currently a required column in epochs

# # Add metadata columns to the epochs table.
# # Note: in Bon data, epochs of type 'sleep' do not contain environment and exposure metadata
# nwbf.add_epoch_column(name='epoch_type', description='type of the epoch (i.e. run, sleep)')
# nwbf.add_epoch_column(name='environment', description='behavioral environment of the epoch (i.e. W-Track A)')
nwbf.add_epoch_column(name='exposure', description='number of exposures to this environment')
nwbf.add_epoch_column(name='task', description='behavioral task for this epoch')
nwbf.add_epoch_column(name='apparatus', description='behavioral apparatus for this epoch')

all_sleep_pos = np.empty(shape=(0, 2))
for epoch_num, task_epoch in task_struct.items():
    # Required columns
    epoch_start, epoch_stop = pos_time_ivls[epoch_num-1]  # start/stop times inferred from pos_struct
    m_per_pixel = pos_struct[epoch_num]['cmperpixel'][0,0]/100  # convert pixels to meters
    
    # Metadata (some not present in 'sleep' epochs)
    # Use these to get the task and apparatus below
    epoch_metadata_keys = task_epoch.keys()
    if 'type' in epoch_metadata_keys:
        epoch_type = task_epoch['type'][0]
    else:
        epoch_type = 'NA'
    if 'environment' in epoch_metadata_keys:
        epoch_env = task_epoch['environment'][0]
    else:
        epoch_env = 'NA'
    if 'exposure' in epoch_metadata_keys:
        epoch_exposure = task_epoch['exposure'][0]
    else:
        epoch_exposure = 'NA'
        
    # Get the Task for this epoch.
    if epoch_type == 'sleep':
        appar = behav_mod.data_interfaces['Sleep Box']
        task = behav_mod.data_interfaces["Sleep"]
    elif epoch_type == 'run':
        task_name = 'W Alternation'
        if task_name not in behav_mod.data_interfaces.keys():   # Create a new Task if necessary
            task_description = 'The animal runs in an alternating W pattern between neighboring arms of a maze.'
            behav_mod.add_data_interface(ap.Task(name=task_name, description=task_description))
        task = behav_mod.data_interfaces[task_name]
    else:  # epoch type not 'sleep' or 'run'
        if epoch_type not in behav_mod.data_interfaces.keys():
            task_description = "Epoch task: {}".format(epoch_type)
            behav_mod.add_data_interface(ap.Task(name=epoch_type, description=task_description))
        task = behav_mod.data_interfaces[task_name]
        
    # Get the Apparatus for this epoch
    if epoch_type == 'sleep':
        appar = behav_mod.data_interfaces['Sleep Box']
    else:
        if epoch_env not in behav_mod.data_interfaces.keys():
            appar = ap.get_apparatus_from_linpos(linpos_struct[epoch_num], epoch_env, conversion=m_per_pixel) 
            behav_mod.add_data_interface(appar)
        appar = behav_mod.data_interfaces[epoch_env]
            
    
    # Add this epoch to the NWB file
    # We include soft link to this task, which itself contains a link to the apparatus
    nwbf.add_epoch(start_time=epoch_start,
                   stop_time=epoch_stop,
                   exposure=epoch_exposure,
                   task=task,
                   apparatus=appar,
                   tags=epoch_tags)



## Tetrode info
Load in `tetinfo` struct and populate ElectrodeTable, electrode groups, etc.

In [39]:
# Create the electrode table.
# The logic here is as follows:
#   Each Tetrode gets its own ElectrodeGroup and ElectrodeTableRegion
#   Each individual recording channel gets its own row in nwbfile.electrodes

# we first create the ElectrodeTable that all the electrodes will go into
nchan_per_tetrode = 4 #these files all contain tetrodes, so we assume four channels
tetinfo_filename = "%s/%s%s" % (data_dir, prefix, tetinfo_file)
recording_device = nwbf.create_device(name='NSpike acquisition system')
tet_electrode_group = {}
tet_electrode_table_region = {}
lfp_electrode_table_region = {}

In [40]:
mat = ns.loadmat_ff(tetinfo_filename, 'tetinfo')
#only look at first epoch because rest are duplicates
tets = mat[day][1]

# For debugging, limit number of tets to import
subset_keys = sorted(tets.keys())[0:limit_num_of_tets]
tets = {k:v for (k,v) in tets.items() if k in subset_keys}

print(limit_num_of_tets)
print("Using tetrode numbers:")
print(subset_keys)

None
Using tetrode numbers:
[1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30]


In [41]:
tets[1]['depth'][0,0][0,0]

149

In [42]:
# kenny's data has a nested [day][epoch][tetrode] structure but duplicates the info across epochs, so we can just
# use the first epoch for everything
chan_num = 0 # this will hold an incrementing channel number for the entire day of data
for tet_num, tet in tets.items():
    #print('making electrode group for day %d, tet %d' % (day, tet_ind))
    # go through the list of fields
    hemisphere = '?'
    # tet.area/.subarea are 1-d arrays of Unicode strings
    area = str(tet['area'][0]) if 'area' in tet else '?' # h5py barfs on numpy.str_ type objects?
    if 'sub_area' in tet: 
        sub_area = str(tet['sub_area'][0]) # h5py barfs on numpy.str_ type objects?
        location = area + ' ' + sub_area
    else:
        sub_area = '?'
        location = area 

    # tet.depth is a 1x1 cell array in tetinfo struct for some reason (multiple depths?)
    # (which contains the expected 1x1 numeric array)
    coord = [np.nan, np.nan, tet['depth'][0, 0][0, 0] / 12 / 80 * 25.4] if 'depth' in tet else [np.nan, np.nan, np.nan]
    impedance = np.nan
    filtering = 'unknown - likely 600Hz-6KHz'

    channel_location = [location, location, location, location]
    channel_coordinates = [coord, coord, coord, coord]
    electrode_name = "%02d-%02d" % (day, tet_num)
    description = "tetrode {tet_num} located in {location} on day {day}".format(tet_num=tet_num,
                                                                               location=location,
                                                                               day=day)

    # we need to create an electrode group for this tetrode
    tet_electrode_group[tet_num] = nwbf.create_electrode_group(name=electrode_name,
                                                        description=description,
                                                        location=location,
                                                        device=recording_device)

    for i in range(nchan_per_tetrode):
            # now add an electrode
            nwbf.add_electrode(x=coord[0],
                               y=coord[1],
                               z=coord[2],
                               imp=impedance,
                               location=location,
                               filtering=filtering,
                               group=tet_electrode_group[tet_num],
                               group_name=tet_electrode_group[tet_num].name, # not in docstring??
                               id=chan_num)
            chan_num = chan_num + 1

    # now that we've created four entries, one for each channel of the tetrode, we create a new
    # electrode table region for this tetrode and number it appropriately
    table_region_description = 'ntrode %d region' % tet_num
    table_region_name = '%d' % tet_num
    tet_electrode_table_region[tet_num] = nwbf.create_electrode_table_region(
        region=list(range(chan_num-nchan_per_tetrode,chan_num)),
        description=table_region_description,
        # BUG #679: name must be 'electrodes' or NWB file will not be readable
        name='electrodes') #        name=table_region_name)



    # Also create electrode_table_regions for each tetrode's LFP recordings
    # (Assume that LFP is taken from the first channel)
    lfp_electrode_table_region[tet_num] = nwbf.create_electrode_table_region(
        region=[chan_num-nchan_per_tetrode],
        description=table_region_description,
        # BUG #679: name must be 'electrodes' or NWB file will not be readable
        name='electrodes') #        name=table_region_name)

In [43]:
# tet_electrode_table_region[1].region
# nwbf.ec_electrode_groups['03-01'].description
# tet_electrode_group

## LFP

In [44]:
%%time

eeg_files = ns.get_eeg_by_day(eeg_path, prefix, 'eeg')
lfp_data = []

lfp = pynwb.ecephys.LFP(electrical_series=lfp_data)
# read data from EEG/*eeg*.mat files and build TimeSeries object

print('processing LFP data for day %2d' % day)
for tet_num in tets.keys():
    print(' -> tet_num: %d' % tet_num)
    timestamps, data = ns.build_day_eeg(eeg_files[day][tet_num], eeg_samprate)
    timestamps += NSpike_posixtime_offset
    name = "{prefix}eeg-{day}-{tet}".format(prefix=prefix, day=day, tet=tet_num)
    lfp.create_electrical_series(name=name, 
                                 data=data / 1000, # convert mV to V, as expected
                                 electrodes=lfp_electrode_table_region[tet_num],
                                 timestamps=timestamps)
nwbf.add_acquisition(lfp)


processing LFP data for day  5
 -> tet_num: 1


KeyError: 5

## Spikes

In [45]:
# Create unit metadata first
# External clustering software gives names for each cluster--we want to preserve these
nwbf.add_unit_column('cluster_name',  '(str) cluster name from clustering software')

# # For tetrode data, this will usually be all channels in the tetrode (2018Dec03--now in spec with 'electrodes' field?)
# nwbf.add_unit_column('neighborhood',  '(electrodeTableRegion) list of electrodes on which spikes were clustered')

# # AKA 'Valid_times'--the times during which a spike from this cluster could have possibly been observed.
# # (handle periods between behavior epochs, acquisition system dropouts, etc.)
# nwbf.add_unit_column('obs_intervals', '(intervalSeries) Observation Intervals for the spike times')

In [46]:
#get the spike times from the spikes files
#each cluster gets a unique number starting at zero

spike_files = ns.get_files_by_day(data_dir, prefix, 'spikes')
print('\nLoading spikes file :' + spike_files[day])
mat = ns.loadmat_ff(spike_files[day], 'spikes')


Loading spikes file :/Users/ericmiller/Data/FrankData/kkay/Bon/bonspikes05.mat


In [47]:
spike_unit = []
obs_intervals = {}
cluster_by_tet = {}
cluster_id = 0

# Matlab structs are nested by: day, epoch, tetrode, cluster, but we will want to save all spikes from a give cluster
# *across multiple epochs* in same spike list. So we rearrange the nested matlab structures for convenience. We 
# create a nested dict, keyed by 1) tetrode, 2) cluster number, then 3) epoch. NB the keys are 1-indexed, to be 
# consistent with the original data collection. (We only process one day at a time for now, so no need to nest days).

spike_struct = mat[day]
for epoch_num, espikes in spike_struct.items():
    for tet_num, tspikes in espikes.items():
        # respect tet subset selection done above
        if tet_num not in tets.keys():
            continue
        if tet_num not in cluster_by_tet.keys():
            cluster_by_tet[tet_num] = {}
        for cluster_num, cspikes in tspikes.items():
            if cluster_num not in cluster_by_tet[tet_num].keys():
                cluster_by_tet[tet_num][cluster_num] = {}
            cluster_by_tet[tet_num][cluster_num][epoch_num] = cspikes
                                

In [48]:
# now we create the SpikeEventStructures and their containing EventWaveform objects
colidx_timestamps = 0

for tet_num in cluster_by_tet.keys():
    obs_intervals[tet_num] = {}
    for cluster_num in cluster_by_tet[tet_num].keys():
        cluster_name = 'd%d t%d c%d' % (day, tet_num, cluster_num)
        print('Adding cluster id: %3d, name: %s' % (cluster_id, cluster_name))

        cluster_tmp = cluster_by_tet[tet_num][cluster_num]

#         # construct a full data array and a parallel list of observation intervals
#         obs_intervals[tet_num][cluster_num] = pynwb.misc.IntervalSeries(
#                                                     name = cluster_name,
#                                                     description = 'Observation intervals for spikes from cluster ' +
#                                                     str(cluster_num) + ' on tetrode ' + str(tet_num))
        obs_intervals[tet_num][cluster_num] = np.zeros([0,2])

        spikes_ep = []
        for epoch in cluster_tmp.keys():
            if cluster_tmp[epoch]['data'].shape[0]:
                spikes_ep.append(cluster_tmp[epoch]['data'][:,colidx_timestamps] + NSpike_posixtime_offset)
            for obs_intervals_cl_ep in cluster_tmp[epoch]['timerange']:
                # 'timerange' for each cell is given in NSpike timestamp units
                obs_int = (obs_intervals_cl_ep.T.astype(float)/NSpike_timestamps_per_sec) + NSpike_posixtime_offset
                obs_intervals[tet_num][cluster_num] = np.append(obs_intervals[tet_num][cluster_num], [obs_int], axis=0)
                #                obs_intervals[tet_num][cluster_num].add_interval(*obs_int)

        spiketimes = np.concatenate(spikes_ep)

        # Add Observation Intervals to nwbfile willy-nilly (1 per cluster), 
        # so that we can successfully refer to them in the Unit metadata table
#        spike_mod.add_data_interface(obs_intervals[tet_num][cluster_num])

#         nwbf.add_unit(data = {'cluster_name': cluster_name, 
#                               'elec_group': tet_electrode_group[tet_num], 
#                               # can't just refer to electrode_table_region itself?: are never added to nwbfile to 
#                               # begin with. Instead, use 'data' field, which is a list of electrodeTable indices.
# #                               'neighborhood': tet_electrode_table_region[tet_num], # tet_electrode_table_region[tet_num].data, 
#                               'obs_intervals': obs_intervals[tet_num][cluster_num]},
#                       id = cluster_id)

#         spike_UnitTimes.add_spike_times(cluster_id, spiketimes)

        nwbf.add_unit(spike_times= spiketimes,
                      electrodes=tet_electrode_table_region[tet_num].data, # wants list of electrode ids, not an electrodeTableRegion
#                       electrode_group=[tet_electrode_group[tet_num]], # we can't figure out what is an acceptable input, here:
#                                                                       # TypeError: incorrect type for 'container' (got 'list', expected 'Builder, Container or ReferenceBuilder')
                      cluster_name=cluster_name, 
                      obs_intervals=obs_intervals[tet_num][cluster_num],
                      id = cluster_id)

        cluster_id += 1

Adding cluster id:   0, name: d5 t1 c1
Adding cluster id:   1, name: d5 t1 c2
Adding cluster id:   2, name: d5 t1 c3
Adding cluster id:   3, name: d5 t1 c4
Adding cluster id:   4, name: d5 t1 c5
Adding cluster id:   5, name: d5 t1 c6
Adding cluster id:   6, name: d5 t1 c7
Adding cluster id:   7, name: d5 t1 c8
Adding cluster id:   8, name: d5 t1 c9
Adding cluster id:   9, name: d5 t2 c1
Adding cluster id:  10, name: d5 t2 c2
Adding cluster id:  11, name: d5 t2 c3
Adding cluster id:  12, name: d5 t2 c4
Adding cluster id:  13, name: d5 t2 c5
Adding cluster id:  14, name: d5 t3 c1
Adding cluster id:  15, name: d5 t3 c2
Adding cluster id:  16, name: d5 t5 c1
Adding cluster id:  17, name: d5 t5 c2
Adding cluster id:  18, name: d5 t5 c3
Adding cluster id:  19, name: d5 t5 c4
Adding cluster id:  20, name: d5 t8 c1
Adding cluster id:  21, name: d5 t8 c2
Adding cluster id:  22, name: d5 t10 c1
Adding cluster id:  23, name: d5 t11 c1
Adding cluster id:  24, name: d5 t11 c2
Adding cluster id:  25

### Write out NWBfile!

In [49]:
# make an NWBFile
with pynwb.NWBHDF5IO(nwb_filename, mode='w') as iow:
    iow.write(nwbf)
print('Wrote nwb file: ' + nwb_filename)

Wrote nwb file: Bon05_test.nwb


In [50]:
print(nwbf.electrodes.id.data)
print(nwbf.units.columns)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107]
(
cluster_name <class 'pynwb.core.VectorData'>
Fields:
  description: (str) cluster name from clustering software
, 
spike_times_index <class 'pynwb.core.VectorIndex'>
Fields:
  target: spike_times <class 'pynwb.core.VectorData'>
, 
spike_times <class 'pynwb.core.VectorData'>
Fields:
  description: the spike times for each unit
, 
obs_intervals_index <class 'pynwb.core.VectorIndex'>
Fields:
  target: obs_intervals <class 'pynwb.core.VectorData'>
, 
obs_intervals <class 'pynwb.core.VectorData'>
Fields:
  description: the observation intervals for each unit
, 
elec

### Read our NWBfile, and check some roundtrip data

In [51]:
io = pynwb.NWBHDF5IO(nwb_filename, mode='r')
nwbf_read = io.read()

In [ ]:
cl_id = 1
print(nwbf_read.units.get_unit_spike_times(cl_id).shape)
print(nwbf_read.units['cluster_name'][cl_id])
print(nwbf_read.units['obs_intervals'][cl_id])
print(nwbf_read.units.get_unit_obs_intervals(cl_id))

In [ ]:
np.where(np.array(nwbf.epochs['epoch_type'][:]) == 'run')[0]

In [ ]:
# Get one of the electrical_series objects and work with it.
example_es_name, example_es = list(nwbf_read.get_acquisition('LFP').electrical_series.items())[-1]
nwbf_read.get_acquisition('LFP').electrical_series

def fmt_sec_from_midnight (x, pos):
    x_posix = mdates.num2epoch(x)
    x_dt = mdates.num2date(x) 
    prev_midnight = x_dt.replace(hour=0, minute=0, second=0, microsecond=0)
    return (x_dt - prev_midnight).total_seconds()
#    return "%0.6f" % mdates.num2epoch(x)

fig1 = plt.figure(1, figsize=(15,10))
ax1 = fig1.add_subplot(1,1,1)
print(ax1.xaxis.major.formatter)
xtick_locator = mticker.AutoLocator()
xtick_formatter = mticker.FuncFormatter(qu.fmt_truncate_posix)
# xtick_formatter = mdates.AutoDateFormatter(xtick_locator)
# xtick_formatter = mticker.ScalarFormatter(xtick_locator)


for i in [1]: #range(len(list(nwbf_read.get_acquisition('LFP').electrical_series.items()))):
    example_es_name, example_es = list(nwbf_read.get_acquisition('LFP').electrical_series.items())[i]
    nwbf_read.get_acquisition('LFP').electrical_series
    #ts = (example_es.timestamps[-1001:-1]*1e9).astype('datetime64[ns]')
    ts = example_es.timestamps[-10001:-1] 
    print("plot #%d" % i)
    ax1.plot(ts, example_es.data[0:10000] * 1000)

ax1.set_title(example_es_name)
ax1.set_xlabel('Time (s)')
# ax1.set_xlabel('Time (s; prefix = 113615\u2026)')
ax1.set_ylabel('Amplitude (mV)')
ax1.xaxis.set_major_locator(xtick_locator)
ax1.xaxis.set_major_formatter(xtick_formatter)
print(ax1.xaxis.major.formatter)


# xtick_locator = mdates.AutoDateLocator()
#fig1.autofmt_xdate()

In [ ]:
io.close()